In [1]:
!pip install transformers

    100% |████████████████████████████████| 665kB 26.1MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 24.0MB/s ta 0:00:01
    100% |████████████████████████████████| 686kB 25.9MB/s ta 0:00:01
    100% |████████████████████████████████| 1.1MB 23.6MB/s ta 0:00:01
    100% |████████████████████████████████| 3.8MB 11.1MB/s ta 0:00:01
    100% |████████████████████████████████| 307kB 33.6MB/s ta 0:00:01
  Running setup.py bdist_wheel for sacremoses ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
fastai 1.0.60 requires nvidia-ml-py3, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import torch
from transformers import BertTokenizer
from transformers import BertModel, BertConfig, BertForSequenceClassification
from transformers import AdamW
from tqdm.autonotebook import tqdm
import pandas as pd
import numpy as np
import boto3
from sagemaker import get_execution_role
from scw import workspace
tqdm.pandas()

In [3]:
bucket = 'a204383-scw-use1-brief-issue'
wksp = workspace.Workspace()
sagemaker_session = wksp.create_session()
role = get_execution_role()

In [4]:
def get_file(s3_file_name, local_filename):
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).download_file(s3_file_name, local_filename)

def save_file(s3_file_name, filename):
    s3 = boto3.resource('s3')
    s3.Bucket(bucket).upload_file(filename, s3_file_name)

In [5]:
# get_file('semi_super_training_data_with_use_vectors_80_with_summary.zip','semi_super_training_data_with_use_vectors_80_with_summary.zip')
# get_file('training_data_with_use_vectors_with_summary.zip','training_data_with_use_vectors_with_summary.zip')

In [6]:
# semi_training_df = pd.read_pickle('semi_super_training_data_with_use_vectors_80_with_summary_new.zip', compression='zip')

In [7]:
labeled_test_df = pd.read_pickle('training_data_with_use_vectors_with_summary_new.zip', compression='zip')
print(len(labeled_test_df))

1571


In [8]:
from sklearn.model_selection import train_test_split
#Split the fuzzy matched records to be mixed in with training, the other half exclusively for testin
train_labeled, test_labeled = train_test_split(labeled_test_df, test_size=0.2)

In [9]:
train_text_list =  train_labeled['summary'].tolist() 
train_labels = np.array(train_labeled['issueArea'].tolist())
test_text_list = test_labeled['summary'].tolist()
test_labels = np.array(test_labeled['issueArea'].tolist())

In [10]:
test_data = list(zip(test_text_list,test_labels))
test_df = pd.DataFrame(test_data, columns=['summary','label'])
test_df.to_pickle('test_data_unseen_bert_without_addl_data.zip', compression='zip')
test_df.head()

,summary,label
0,"[{'summary_text': 'BULLOCK v. South Carolina, ...",9.0
1,[{'summary_text': 'Petitioner was convicted of...,1.0
2,[{'summary_text': 'Robert Toussie was convicte...,2.0
3,[{'summary_text': 'Petitioner was convicted of...,9.0
4,[{'summary_text': 'United States Supreme Court...,1.0


In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix

In [12]:
BERT_MODEL = 'bert-base-cased'
num_labels = 14
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, do_lower_case=False)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels=num_labels)

In [13]:
device = torch.device('cuda')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [14]:
# Default MAX sequence length for BertModel
MAX_SEQ_LENGTH = 256
def pad_sequences(pad_token, seq_list, max_length):
    return seq_list + [pad_token] * (max_length - len(seq_list))

def pad_special_tokens(tokenized_text_sent):
    if len(tokenized_text_sent) > MAX_SEQ_LENGTH - 2:
           tokenized_text_sent = tokenized_text_sent[0:(MAX_SEQ_LENGTH - 2)]            
    tokenized_text_sent.insert(0,'[CLS]')
    tokenized_text_sent.append('[SEP]')
    return tokenized_text_sent

def tokenize_sentence(summary_text): 
    tokenized_text_sent = tokenizer.tokenize(summary_text[0]['summary_text'])                  
    tokenized_text_sent = pad_special_tokens(tokenized_text_sent)
    return tokenized_text_sent  

tokenized_seq = list(map(tokenize_sentence, train_text_list))
print(tokenized_seq[0])

['[CLS]', 'Petit', '##ione', '##r', 'was', 'convicted', 'in', 'Federal', 'District', 'Court', 'of', 'violating', ',', 'inter', 'al', '##ia', ',', '231', '##4', ',', 'arising', 'from', 'the', 'interstate', 'transportation', 'of', '"', 'boot', '##leg', '"', 'p', '##hon', '##ore', '##cor', '##ds', 'that', 'were', 'manufactured', 'and', 'distributed', 'without', 'the', 'consent', 'of', 'the', 'copyright', 'owners', 'of', 'the', 'musical', 'compositions', 'performed', 'on', 'the', 'records', '.', 'The', 'language', 'of', '231', '##4', 'does', 'not', '"', 'plain', '##ly', 'and', 'un', '##mist', '##aka', '##bly', '"', 'cover', 'such', 'conduct', '.', 'In', '##f', '##ring', '##ement', 'imp', '##lica', '##tes', 'a', 'more', 'complex', 'set', 'of', 'property', 'interests', 'than', 'does', 'run', '-', 'of', '-', 'the', '-', 'mill', 'theft', ',', 'conversion', ',', 'or', 'fraud', '.', 'The', 'history', 'of', 'the', 'criminal', 'infringement', 'provisions', 'of', 'the', 'Copyright', 'Act', 'indicat

In [15]:
def convert_tokens_to_tensor(tokenized_sentence_a):        
    tokenized_text = []
    tokenized_text.extend(tokenized_sentence_a)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    indexed_tokens = pad_sequences(0, indexed_tokens, MAX_SEQ_LENGTH)    
    #tokens_tensor = torch.tensor([indexed_tokens])
    #generate the token type ids
    token_type_ids = []
    token_type_a = [0] * len(tokenized_sentence_a)
    token_type_ids.extend(token_type_a)
    token_type_ids = pad_sequences(0, token_type_ids, MAX_SEQ_LENGTH)    
    #token_type_tensor = torch.tensor([token_type_ids])
    #generate the type ids
    input_mask = [1] * len(tokenized_text)
    input_mask = pad_sequences(0, input_mask, MAX_SEQ_LENGTH)    
    #input_tensor = torch.tensor([input_mask])
    return [indexed_tokens, token_type_ids, input_mask]    

converted_tensors = list(map(convert_tokens_to_tensor, tokenized_seq))
print(converted_tensors[0])

[[101, 21386, 23997, 1197, 1108, 7590, 1107, 3467, 1574, 2031, 1104, 22128, 117, 9455, 2393, 1465, 117, 22154, 1527, 117, 19528, 1121, 1103, 20905, 6312, 1104, 107, 10909, 27412, 107, 185, 8613, 4474, 19248, 3680, 1115, 1127, 7227, 1105, 4901, 1443, 1103, 9635, 1104, 1103, 11409, 5032, 1104, 1103, 2696, 9116, 1982, 1113, 1103, 3002, 119, 1109, 1846, 1104, 22154, 1527, 1674, 1136, 107, 6188, 1193, 1105, 8362, 18972, 11747, 4999, 107, 2267, 1216, 5880, 119, 1130, 2087, 3384, 14529, 24034, 9538, 3052, 170, 1167, 2703, 1383, 1104, 2400, 4740, 1190, 1674, 1576, 118, 1104, 118, 1103, 118, 6159, 14284, 117, 7497, 117, 1137, 10258, 119, 1109, 1607, 1104, 1103, 4771, 23040, 8939, 1104, 1103, 22359, 2173, 6653, 1115, 2757, 1125, 1185, 6247, 1106, 2519, 11409, 23040, 1165, 1122, 12113, 22154, 1527, 119, 1302, 1216, 1444, 1111, 15491, 1348, 2877, 2168, 1144, 1518, 5131, 1114, 4161, 1106, 11409, 23040, 117, 1290, 2757, 1144, 1103, 1540, 1223, 1103, 5317, 1106, 3420, 1548, 8052, 2626, 1107, 1142, 12

In [16]:
def one_hot_embedding(labels, num_classes):
    """Embedding labels to one-hot form.

    Args:
      labels: (LongTensor) class labels, sized [N,].
      num_classes: (int) number of classes.

    Returns:
      (tensor) encoded labels, sized [N, #classes].
    """
    y = torch.eye(num_classes) 
    return y[labels] 

all_input_ids = torch.tensor([f[0] for f in converted_tensors])
all_input_mask = torch.tensor([f[2]for f in converted_tensors])
all_segment_ids = torch.tensor([f[1] for f in converted_tensors])
all_labels = torch.tensor(train_labels, dtype=torch.long)
# one_hot_train_labels = one_hot_embedding(train_labels, num_labels)
# all_labels = torch.tensor(one_hot_train_labels, dtype=torch.long)

# print(all_input_ids.size(0))
# print(all_input_mask.size(0))
# print(all_segment_ids.size(0))
#print(all_labels.size())


In [17]:
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler, TensorDataset)

In [18]:
import math
MAX_GRAD_NORM = 1.0
NUM_TRAIN_EPOCHS = 20
TRAIN_BATCH_SIZE = 10
train_examples_len = len(all_input_ids)
num_train_optimization_steps = math.ceil((train_examples_len / TRAIN_BATCH_SIZE)) * NUM_TRAIN_EPOCHS
LEARNING_RATE = 2e-5
num_warmup_steps=100

In [19]:
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=TRAIN_BATCH_SIZE)

In [20]:
print(num_train_optimization_steps)

2520


In [21]:
# param_optimizer = list(model.named_parameters())
# no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
# optimizer_grouped_parameters = [
#     {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
#     {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
#     ]

In [22]:
from transformers import get_linear_schedule_with_warmup
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE,  correct_bias=False)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_train_optimization_steps)

In [23]:
print("***** Running training *****")
print("Num examples = {}".format(train_examples_len))
print("Batch size = {}".format(TRAIN_BATCH_SIZE))
print("Num steps = {}".format(num_train_optimization_steps))

***** Running training *****
Num examples = 1256
Batch size = 10
Num steps = 2520


In [24]:
global_step = 0
nb_tr_steps = 0
tr_loss = 0

In [25]:
from tqdm import tqdm_notebook, trange
from torch.nn import CrossEntropyLoss

model.train()
for _ in trange(int(NUM_TRAIN_EPOCHS), desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(tqdm_notebook(train_dataloader, desc="Iteration")):
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        outputs, logits = model(input_ids, token_type_ids=segment_ids, attention_mask=input_mask,labels=label_ids)
        loss_fct = CrossEntropyLoss() 
        loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)
        print("\r%f" % loss, end='')        
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        if (step + 1) % 1 == 0:
            optimizer.step()
            optimizer.zero_grad()
            scheduler.step()
            global_step += 1

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


1.505422

Epoch:   5%|▌         | 1/20 [01:58<37:28, 118.36s/it]

0.940146


1.429197

...

Epoch:  95%|█████████▌| 19/20 [35:28<01:51, 111.42s/it]

0.001365


0.000913

Epoch: 100%|██████████| 20/20 [37:19<00:00, 111.98s/it]

0.001313


In [26]:
import os
from transformers import WEIGHTS_NAME, CONFIG_NAME
# # Save a trained model
model_out = 'scdb_train_tuned_model_without_addl_data'
#os.mkdir(model_out)
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(model_out, WEIGHTS_NAME)
output_config_file = os.path.join(model_out, CONFIG_NAME)
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
vocab = tokenizer.save_vocabulary(model_out)